### Introduction:

The IceCube Neutrino Observatory is composed of many light detectors, called "DOMs" (digital optical modules), buried two kilometers deep below the surface of the South Pole. These light detectors are arranged in a roughly three dimensional grid: there are many "strings" hanging vertically downwards in the ice, and on each string there are many DOMs: 

![](../resources/images/I3det_v2_edited.jpeg)

When a high energy charged particle passes through the ice, it produces what is called Cherenkov light, which can then be detected by the many DOMs. The **locations** of the DOMs that see light, the **times** at which they saw light, and the **amount** of light they see all communicate information about this charged particle.

As experimentalists, what we want to do is work backwards: if what we have is this information about the light seen by the detector, what can we figure out about the original particle? 

### 1: Visualizing events:

For this activity, we've prepared a simulation of many events within the IceCube detector. Execute the cell below by clicking the triangle or hitting `shift return` to download the simulation from google drive.

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
from IPython.display import display, clear_output

# download tracks.parquet
!gdown 1nqffw6xHLdX2oO8d-5xjExYZryo7rp3x

# download cascades.parquet
!gdown 1yo3jD0a9xB2FfIXJJuMRc71IMe6nztvq

# download pre-prepared analysis code
!rm -rf IceCube_MasterClass_at_Harvard2024
!git clone "https://github.com/kcarloni/IceCube_MasterClass_at_Harvard2024";

import sys
sys.path.insert(0, "./IceCube_MasterClass_at_Harvard2024/")

In [ ]:
from src.event_reader import EventSelection, Event
tracks = EventSelection("tracks.parquet")

Let's look at an 'event' in the detector. An event is just a bunch of 'hits', ie. instances in which light hit different sensors, which are grouped together because they all occured within the same time window.

In [ ]:
evt_num = 24
evt = tracks[evt_num]
evt

We can visualize an event within the detector by plotting it.  

In [ ]:
import plotly.graph_objs as go
from src.plot_event import *

layout = get_3d_layout()
plot_det = plot_I3det()

fig = go.FigureWidget(data=plot_det, layout=layout)

plot_evt = plot_first_hits(evt)
fig.add_trace(plot_evt)

fig.show()

### 2: From what direction did the neutrino come from? 

In the event displays we've just looked at, redder hits occur earlier and greener hits occur after. Given this information, can you figure out in what direction the neutrino was going?

We've written a game below to see how well you can do. Execute the cell below (`shift return`) to start the game. 

You should see a similar event display, along with a big arrow, which represents your guess for the direction of the neutrino. Adjust this arrow using the zenith and azimuth angle sliders provided. Once you are happy with your guess, hit the submit button below the event display. The event display will be refreshed to show the actual true path of the neutrino, and how many degrees you were off the actual path will be printed below. Try to be as close as possible to the actual path, and minimize the degrees you were off by!

After submitting a guess, you can return to the game by clicking the Return button. You can then try to guess again on the same event, or try your hand at a different one by opening the event_id dropdown menu and selecting a different one.

WIP: Felix's directional reconstruction game. 

In [ ]:
!pip install "analysis @ git+https://github.com/jlazar17/IceCube_Masterclass_MoonShadow";

from analysis import DataReader
from analysis import reco_game

tracks = DataReader("tracks.parquet")

In [ ]:
reco_game(tracks)

As we discussed earlier today, IceCube can see two broad kinds of events, *tracks*, which look like long lines and are produced by the light from *muons*, and *cascades*, which look like spherical blobs and are produced by the light from *electrons* (and some other particles). 

We just tried to reconstruct the direction of some track events. Do you think it would be easier or harder to reconstruct the direction of some cascade events? 

Give it a try!

In [ ]:
cascades = DataReader("cascades.parquet")
reco_game(cascades)

### 3: Can we use computers to do a better job at figuring out the direction?

When we try to figure out the direction in the game below, what we're actually doing is trying to get the direction arrow to go as close to through the middle of all the hits as possible.

We can quantify this sense of "how good" with some math! Imagine drawing straight lines from each hit to the arrow, like in the image below:

the more of these lines are shorter, the better our guess of the direction. We can calculate the length of each line using some *vector* math:

We can also write a *function* in python to do this calculation for us! Just like in math, this *function* takes in some inputs (the direction vector and the hit) and produces one output (the perpendicular distance). 

In [ ]:
def calc_perpendicular_distance_squared( hit_pt, dir_vec, pivot_pt ):
    
    dist_vec = hit_pt - pivot_pt
    return np.dot( dist_vec, dist_vec ) -  np.dot( dist_vec, dir_vec )**2


def get_direction_vector_from_angles( azi, zen ):
    
    return np.array([
        np.cos(azi) * np.sin(zen),
        np.sin(azi) * np.sin(zen),
        np.cos(zen)
    ])

def calc_center_of_gravity(evt):
    
    hits = evt.hits[["t", "sensor_pos_x", "sensor_pos_y", "sensor_pos_z"]].to_numpy()
    return hits.mean(axis=0)[1:4]

Let's check that bad guesses really result in larger values for the perpendicular distance. Modify the angles below, and check how bad the guess is: 

In [ ]:
zen = 
azi = 

dir_vec = get_direction_vector_from_angles( azi, zen )
pivot_pt = calc_center_of_gravity(evt)

fig_with_arrow = go.FigureWidget( fig )
fig_with_arrow.add_traces( plot_direction( dir_vec, pivot_pt, color="dodgerblue" ) )
fig_with_arrow.show()

WIP: implement an optimizer-based directional reco?

WIP (?) implement / load some kind of ML reco to show? 

#### 4. How can we quantify the quality of our "reconstructed" directions?

WIP: make plots to show how different reconstructions compare; performance as a function of energy...